Classify this scientific image into one of the following categories:

TABLES/CHARTS:
- Data table
- Statistical table
- Comparison table

GRAPHS/PLOTS:
- Line graph
- Bar chart
- Scatter plot
- Box plot
- Histogram
- Pie chart
- Heat map
- Network graph
- Time series

MICROSCOPY:
- Light microscopy
- Electron microscopy
- Fluorescence microscopy
- Confocal microscopy
- Super-resolution microscopy

SPECTROSCOPY:
- Mass spectroscopy
- NMR spectroscopy
- IR spectroscopy
- UV-vis spectroscopy
- X-ray spectroscopy

MEDICAL/BIOLOGICAL IMAGING:
- X-ray
- CT scan
- MRI scan
- Ultrasound
- PET scan
- Histology slide
- Western blot
- Gel electrophoresis

DIAGRAMS/ILLUSTRATIONS:
- Chemical structure
- Molecular diagram
- Anatomical illustration
- Flowchart
- Schematic diagram
- Circuit diagram
- Mechanical diagram
- Process flow diagram

MAPS/GEOGRAPHICAL:
- Geographic map
- GIS visualization
- Satellite image
- Terrain model

MATHEMATICAL:
- Equation
- Mathematical model
- Geometric figure
- Mathematical plot

COMPUTER-GENERATED:
- 3D rendering
- Simulation visualization
- Computer model

MISCELLANEOUS:
- Field photograph
- Sample photograph
- Experimental setup
- Equipment photograph
- Screenshot
- Logo/Institutional insignia
- Cover art
- Author photograph
- Infographic
- Conceptual illustration



In [8]:
IMAGE_CLASSIFICATION_SYSTEM_PROMPT = """
You are an expert image classifier specializing in scientific literature. Your task is to classify images from academic papers and research documents into precise numbered categories. Analyze each image carefully, considering its visual characteristics, content, and typical usage in scientific literature.

Return your analysis as a valid JSON object with exactly this format:
{
  "class": int,
  "confidence": float
}

Where "class" is the integer corresponding to the image category, and "confidence" is a number between 0.0 and 1.0 representing your confidence in the classification. Do not include any other fields in your response - only these two fields in this exact format.
"""
IMAGE_CLASSIFICATION_USER_PROMPT = """
Classify this scientific image into exactly one of the following numbered categories:

1. Data table
2. Statistical table
3. Line graph
4. Bar chart
5. Scatter plot
6. Box plot
7. Histogram
8. Pie chart
9. Heat map
10. Network graph
11. Time series plot
12. Light microscopy
13. Electron microscopy
14. Fluorescence microscopy
15. Confocal microscopy
16. Mass spectroscopy
17. NMR spectroscopy
18. IR spectroscopy
19. UV-vis spectroscopy
20. X-ray spectroscopy
21. X-ray (medical)
22. CT scan
23. MRI scan
24. Ultrasound
25. PET scan
26. Histology slide
27. Western blot
28. Gel electrophoresis
29. Chemical structure
30. Molecular diagram
31. Anatomical illustration
32. Flowchart
33. Schematic diagram
34. Circuit diagram
35. Mechanical diagram
36. Process flow diagram
37. Geographic map
38. GIS visualization
39. Satellite image
40. Equation/Mathematical expression
41. Geometric figure
42. 3D rendering
43. Computer simulation
44. Field photograph
45. Sample photograph
46. Experimental setup
47. Equipment photograph
48. Screenshot
49. Logo/Institutional insignia
50. Infographic
51. other

Return only a JSON with the class number (integer) and your confidence (float between 0.0 and 1.0) in this exact format:
{
  "class": int,
  "confidence": float
}
"""

In [1]:
from openai import AzureOpenAI, OpenAI
from azureml.rag.utils.connections import get_connection_by_id_v2
from azureml.rag.utils.logging import get_logger, safe_mlflow_start_run, track_activity
import os

logger = get_logger("document_analyzer")

def setup_openai_client(connection_id: str, type: str = "openai"):
    """Set up Azure OpenAI client using connection."""
    try:
        # Get connection details
        connection = get_connection_by_id_v2(connection_id)

        # Safely access endpoint and api_key from dictionary
        if connection.endpoint is not None:
            endpoint = connection.endpoint
        else:
            endpoint = connection.get("endpoint") or connection.get("properties", {}).get("metadata", {}).get("endpoint")
        if connection.api_key is not None:
            api_key = connection.api_key
        else:
            api_key = connection.get("api_key") or connection.get("properties", {}).get("credentials", {}).get("keys", {}).get("api_key")

        if not endpoint or not api_key:
            raise ValueError("Missing endpoint or api_key in connection details.")

        if type == "openai":
            # Create client for openai
            client = OpenAI(
                api_key=api_key,
                base_url=endpoint
            )
        else:                
            # Create client for  vision
            client = AzureOpenAI(
                api_key=api_key,
                api_version="2025-01-01-preview",
                azure_endpoint=endpoint
            )

        return client

    except Exception as e:
        logger.error(f"Failed to setup Azure OpenAI connection: {str(e)}")
        raise

In [2]:
# Example of registering the component in a workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Get workspace
ml_client = MLClient.from_config(
    credential=DefaultAzureCredential()
)

Found the config file in: /config.json


In [43]:
aoai_connection_name = "open_ai_connection_01"
# aoai_connection_name = "deepseek"
acs_connection_name = "acs-connection"
data_set_name = "papers"
asset_name = "aoai_acs_mlindex"
doc_intelligence_connection_name = "doc-intelligence-connection"
vision_deploy_name = "gpt-4o"
aoai_embedding_model_name = "text-embedding-3-large"

acs_config = {
    "index_name": "qknows-embedding",
}

experiment_name = "sample-acs-embedding"

aoai_connection_id = ml_client.connections.get(aoai_connection_name).id

aoai_cleint = setup_openai_client(aoai_connection_id, "azure-openai")

# Define your deployment name for the vision model (e.g., "gpt-4v")
vision_deployment_name = "gpt-4o"

In [21]:
url = aoai_cleint.base_url
str(url)

'https://ai-deepseektest505679245816600.openai.azure.com/openai/'

In [5]:
# chat_completion = aoai_cleint.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model="deepseek-r",
# )

# print(chat_completion)

In [6]:
# client.get_model_info(
#     model_name="gpt-4o",
#     deployment_name=vision_deployment_name,
# )

In [26]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.identity import DefaultAzureCredential

url = "https://ai-deepseektest505679245816600.openai.azure.com/"
api_key = aoai_cleint.api_key

endpoint = str(url)
model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential(),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="I am going to Paris, what should I see?"),
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=model_name
)

print(response.choices[0].message.content)

ResourceNotFoundError: (404) Resource not found
Code: 404
Message: Resource not found

In [ ]:
# pip install azure-ai-inference
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
connection = get_connection_by_id_v2(aoai_connection_id)

# Safely access endpoint and api_key from dictionary
if connection.endpoint is not None:
    endpoint = connection.endpoint
else:
    endpoint = connection.get("endpoint") or connection.get("properties", {}).get("metadata", {}).get("endpoint")
if connection.api_key is not None:
    api_key = connection.api_key
else:
    api_key = connection.get("api_key") or connection.get("properties", {}).get("credentials", {}).get("keys", {}).get("api_key")

endpoint = "https://ai-deepseektest505679245816600.openai.azure.com/openai/deployments/gpt-4o"

if not api_key:
  raise Exception("A key should be provided to invoke the endpoint")

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(api_key),
    api_version="2024-06-01",
)

# model_info = client.get_model_info()
# print("Model name:", model_info.model_name)
# print("Model type:", model_info.model_type)
# print("Model provider name:", model_info.model_provider_name)

payload = {
  "messages": [
    {
      "role": "user",
      "content": "I am going to Paris, what should I see?"
    },
    {
      "role": "assistant",
      "content": "Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world."
    },
    {
      "role": "user",
      "content": "What is so great about #1?"
    }
  ],
  "max_tokens": 2048
}
response = client.complete(payload)

print("Response:", response.choices[0].message.content)
print("Model:", response.model)
print("Usage:")
print("	Prompt tokens:", response.usage.prompt_tokens)
print("	Total tokens:", response.usage.total_tokens)
print("	Completion tokens:", response.usage.completion_tokens)

Response: The **Eiffel Tower** is often considered the crown jewel of Paris and one of the most famous landmarks in the world. Its greatness comes from several aspects:

### 1. **Iconic Symbol of Paris and France**  
   - When people think of Paris, the Eiffel Tower is usually the first thing that comes to mind. It’s a universal symbol of romance, elegance, and the city’s artistic and architectural achievements.

### 2. **Unique Architectural Design**
   - Designed by engineer **Gustave Eiffel**, the tower is a masterpiece of iron artistry and engineering. Built for the **1889 Exposition Universelle (World’s Fair)** to celebrate the 100th anniversary of the French Revolution, its revolutionary design was initially controversial but later became beloved worldwide.
   - At its completion, the Eiffel Tower was the **tallest man-made structure in the world** (standing at 330 meters/1,083 feet today with antennas).

### 3. **Incredible Views**
   - Visitors can take an elevator or climb the

In [48]:
import sys

sys.path.append('../aml_pipeline/chuck_caption_component/')
# enhanced_doc_analyzer_component/enhanced_document_analyzer


from openai import AzureOpenAI


from src import DocumentProcessor
from src.prompt import IMAGE_CLASSIFICATION_SYSTEM_PROMPT, IMAGE_CLASSIFICATION_USER_PROMPT

In [49]:
import json


class ImageClassification(DocumentProcessor):
    def __init__(self, input_folder, output_folder, openai_client, vision_deployment_name):
        super().__init__(input_folder, output_folder, openai_client, vision_deployment_name)
        self.vision_client = openai_client
        self.vision_deployment_name = vision_deployment_name
    
    def get_image_classification(self, image_path):
        try:
            # Get base64 encoded image
            base64_image = self.encode_image(image_path)
            if not base64_image:
                print(f"Skipping classification for invalid image: {image_path}")
                return None
            
            # Create ChatCompletionsClient with connection information
            connection = get_connection_by_id_v2(aoai_connection_id)
            
            # Safely access endpoint and api_key from dictionary
            if connection.endpoint is not None:
                endpoint = connection.endpoint
            else:
                endpoint = connection.get("endpoint") or connection.get("properties", {}).get("metadata", {}).get("endpoint")
            if connection.api_key is not None:
                api_key = connection.api_key
            else:
                api_key = connection.get("api_key") or connection.get("properties", {}).get("credentials", {}).get("keys", {}).get("api_key")
            
            # Create endpoint URL with deployment name
            endpoint_url = f"{endpoint}/openai/deployments/{self.vision_deployment_name}"
            
            # Create chat client
            client = ChatCompletionsClient(
                endpoint=endpoint_url,
                credential=AzureKeyCredential(api_key),
                api_version="2025-01-01-preview"
            )
            
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    # Create the payload with system and user messages
                    payload = {
                        "messages": [
                            {
                                "role": "system",
                                "content": IMAGE_CLASSIFICATION_SYSTEM_PROMPT
                            },
                            {
                                "role": "user",
                                "content": [
                                    {
                                        "type": "text",
                                        "text": IMAGE_CLASSIFICATION_USER_PROMPT
                                    },
                                    {
                                        "type": "image_url",
                                        "image_url": {
                                            "url": f"data:image/jpeg;base64,{base64_image}"
                                        }
                                    }
                                ]
                            }
                        ],
                        "temperature": 0.0,
                        "max_tokens": 100,
                        "response_format": {"type": "json_object"}
                    }
                    
                    # Call the API
                    response = client.complete(payload)

                    print("Model:", response.model)
                    print("Usage:")
                    print("	Prompt tokens:", response.usage.prompt_tokens)
                    print("	Total tokens:", response.usage.total_tokens)
                    print("	Completion tokens:", response.usage.completion_tokens)


                    # Extract the JSON response
                    result = json.loads(response.choices[0].message.content)
                    return result
                    
                except json.JSONDecodeError:
                    # If the response isn't valid JSON, try to extract JSON from the text
                    content = response.choices[0].message.content
                    try:
                        # Try to find JSON within the response
                        json_start = content.find('{')
                        json_end = content.rfind('}') + 1
                        if json_start >= 0 and json_end > json_start:
                            json_str = content[json_start:json_end]
                            return json.loads(json_str)
                        else:
                            print(f"Could not extract JSON from response: {content}")
                            if attempt < max_retries - 1:
                                continue
                            return None
                    except Exception as json_extract_err:
                        print(f"Failed to parse JSON response: {str(json_extract_err)}")
                        if attempt < max_retries - 1:
                            continue
                        return None
                
                except Exception as e:
                    print(f"Image classification attempt {attempt + 1} failed: {str(e)}")
                    if attempt < max_retries - 1:
                        continue
                    else:
                        print(f"All image classification attempts failed for {image_path}")
                        return None
                        
        except Exception as e:
            print(f"Image classification failed: {str(e)}")
            return None

In [51]:
# Initialize the image classification service
classifier = ImageClassification(
    input_folder= "/home/azureuser/academic-document-analyzer/docs/",
    output_folder= ".",
    openai_client=aoai_cleint,
    vision_deployment_name=vision_deployment_name
)

# Path to the image you want to classify
image_path = "/home/azureuser/academic-document-analyzer/docs/indexer_pipeline-next-step.png"

# Classify the image
result = classifier.get_image_classification(image_path)

# Process the result
if result:
    class_id = result["image_class"]
    confidence = result["probability_score"]
    
    # Map class ID to class name (you would have this mapping defined somewhere)
    class_names = {
        1: "Data table",
        2: "Statistical table",
        3: "Line graph",
        4: "Bar chart",
        5: "Scatter plot",
        6: "Box plot",
        7: "Histogram",
        8: "Pie chart",
        9: "Heat map",
        10: "Network graph",                
        11: "Time series plot",
        12: "Light microscopy",
        13: "Electron microscopy",
        14: "Fluorescence microscopy",
        15: "Confocal microscopy",
        16: "Mass spectroscopy",
        17: "NMR spectroscopy",
        18: "IR spectroscopy",
        19: "UV-vis spectroscopy",
        20: "X-ray spectroscopy",
        21: "X-ray (medical)",
        22: "CT scan",
        23: "MRI scan",
        24: "Ultrasound",
        25: "PET scan",
        26: "Histology slide",
        27: "Western blot",
        28: "Gel electrophoresis",
        29: "Chemical structure",
        30: "Molecular diagram",
        31: "Anatomical illustration",
        32: "Flowchart",
        33: "Schematic diagram",
        34: "Circuit diagram",
        35: "Mechanical diagram",
        36: "Process flow diagram",
        37: "Geographic map",
        38: "GIS visualization",    
        39: "Satellite image",
        40: "Equation/Mathematical expression",
        41: "Geometric figure",
        42: "3D rendering",     
        43: "Computer simulation",
        44: "Field photograph",
        45: "Sample photograph",
        46: "Experimental setup",
        47: "Equipment photograph",
        48: "Screenshot",
        49: "Logo/Institutional insignia",
        50: "Infographic",
        51: "other"
    }
    
    class_name = class_names.get(class_id, "Unknown class")
    
    print(f"Image classified as: {class_name} (ID: {class_id})")
    print(f"Confidence: {confidence:.2f}")
else:
    print("Classification failed")

2025-03-09 07:28:56,448 - src.document_processor - INFO - Input folder: /home/azureuser/academic-document-analyzer/docs
2025-03-09 07:28:56,449 - src.document_processor - INFO - Output folder: .
2025-03-09 07:28:56,450 - src.document_processor - INFO - Successfully initialized tiktoken tokenizer
Model: gpt-4o-2024-11-20
Usage:
	Prompt tokens: 1004
	Total tokens: 1025
	Completion tokens: 21
Image classified as: Flowchart (ID: 32)
Confidence: 0.95
